In [1]:
# Temporary notebook to rough out scripting for
# make-district-file-2025.py
# Will transfer finished code there and delete this

import math
import json
import pandas as pd

'''
# Aiming for this election result form
# will need to do some logic to determine which year's primary and general elex
# to pull from for senate districts based on whether they were in cycle in 2022
        "pri_elex": {
            "leg": [
                {
                    "name": "STEVE GUNDERSON",
                    "party": "R",
                    "votes": 2121
                },
                {
                    "name": "MARVIN C SATHER",
                    "party": "D",
                    "votes": 880
                }
            ]
        },
        "gen_elex": {
            "gov": [
                {
                    "name": "Gianforte",
                    "party": "R",
                    "votes": 3568
                },
                {
                    "name": "Cooney",
                    "party": "D",
                    "votes": 1499
                },
                {
                    "name": "Bishop",
                    "party": "L",
                    "votes": 210
                }
            ],
            "leg": [
                {
                    "name": "STEVE GUNDERSON",
                    "party": "R",
                    "votes": 3721
                },
                {
                    "name": "MARVIN C SATHER",
                    "party": "D",
                    "votes": 1518
                }
            ]
        },
'''

'\n# Aiming for this election result form\n# will need to do some logic to determine which year\'s primary and general elex\n# to pull from for senate districts based on whether they were in cycle in 2022\n        "pri_elex": {\n            "leg": [\n                {\n                    "name": "STEVE GUNDERSON",\n                    "party": "R",\n                    "votes": 2121\n                },\n                {\n                    "name": "MARVIN C SATHER",\n                    "party": "D",\n                    "votes": 880\n                }\n            ]\n        },\n        "gen_elex": {\n            "gov": [\n                {\n                    "name": "Gianforte",\n                    "party": "R",\n                    "votes": 3568\n                },\n                {\n                    "name": "Cooney",\n                    "party": "D",\n                    "votes": 1499\n                },\n                {\n                    "name": "Bishop",\n        

In [2]:
statewide_general_by_precinct_2024 = pd.read_csv('./inputs/districts/election-results/2024-general-statewide-precinct-vote-counts.csv')
lege_primary_2024 = pd.read_csv('./inputs/districts/election-results/2024-primary-legislative-vote-counts.csv')
lege_general_2024 = pd.read_csv('./inputs/districts/election-results/2024-general-legislative-vote-counts.csv')
lege_primary_2022 = pd.read_csv('./inputs/districts/election-results/2022-primary-legislative-vote-counts.csv')
lege_general_2022 = pd.read_csv('./inputs/districts/election-results/2024-general-legislative-vote-counts.csv')

def parse_legislative_votes(results, district):
    race_votes = results[results['district'] == district][['party','candidate','votes']]
    race_votes['party'] = race_votes['party'].str[:1]
    total_votes = race_votes['votes'].sum()
    race_votes['percent'] = race_votes['votes'] / total_votes
    return race_votes.to_dict(orient='records')

district = 'HD-2'
in_cycle_2024 = True
pri_elex = {
    'leg': parse_legislative_votes(lege_primary_2024, district) if in_cycle_2024 \
        else  parse_legislative_votes(lege_primary_2022, district)
}
gen_elex = {
    'pres': {},
    'gov': {},
    'leg': parse_legislative_votes(lege_general_2024, district) if in_cycle_2024 \
        else  parse_legislative_votes(lege_general_2022, district),
}

In [171]:
# Build a senate/house district precinct-to-district map
# That can be used to pull statewide vote tallies for each district
# Note this is complicated by a few rural precincts that count votes for multiple lege districts
# With those, we're allocating statewide race votes proportionally to the share of MT house votes in each district

def hd_to_sd(key):
    # Each SD is two HD
    hd_number = int(key[3:])
    sd_number = math.ceil(hd_number / 2)
    return f'SD {sd_number}'

# def make_precinct_keys(precinct_results):
#     # Make by-HD and by-SD tables to apportion precinct-level results to specific districts
#     # vote_fraction field accomodates the handful of rural precincts that don't align neatly with legislative districts
#     # We assume votes in statewide races are allocated there proportionaly to state house races

#     lege_votes_by_precinct = precinct_results[precinct_results['race'].str[:2] == 'HD'].copy()
#     lege_votes_by_precinct['hd'] = lege_votes_by_precinct['race']
#     lege_votes_by_precinct['sd'] = lege_votes_by_precinct['race'].apply(hd_to_sd) 

#     precinct_total_votes = lege_votes_by_precinct.groupby('key').agg({'votes': 'sum'}).to_dict()['votes']

#     hd_precinct_key = lege_votes_by_precinct.groupby([
#         'key','county','precinct','hd'
#     ]).agg({'votes': 'sum'}).reset_index()
#     hd_precinct_key['vote_fraction'] = hd_precinct_key.apply(lambda row: row['votes'] / (precinct_total_votes[row['key']] or 1), axis=1)

#     sd_precinct_key = lege_votes_by_precinct.groupby([
#         'key','county','precinct','sd'
#     ]).agg({'votes': 'sum'}).reset_index()
#     sd_precinct_key['vote_fraction'] = sd_precinct_key.apply(lambda row: row['votes'] / (precinct_total_votes[row['key']] or 1), axis=1)

#     return hd_precinct_key, sd_precinct_key

# hd_precinct_key, sd_precinct_key = make_precinct_keys(statewide_general_by_precinct_2024)

In [202]:
def apportion_precinct_results_to_legislative_districts(precinct_results):
    # Create data tables that map specific precincts to their corresponding legislative districts
    # vote_fraction field accomodates the handful of rural precincts that don't align neatly with legislative districts
    # We assume votes in statewide races are allocated there proportionaly to state house races
    # vote_fraction is 1.0 for the vast majority of precints, which map to a single district

    # This could probably be made more elegant

    lege_votes_by_precinct = precinct_results[precinct_results['race'].str[:2] == 'HD'].copy()
    lege_votes_by_precinct['hd'] = lege_votes_by_precinct['race']
    lege_votes_by_precinct['sd'] = lege_votes_by_precinct['race'].apply(hd_to_sd) 

    precinct_total_votes = lege_votes_by_precinct.groupby('key').agg({'votes': 'sum'}).to_dict()['votes']

    hd_precinct_key = lege_votes_by_precinct.groupby([
        'key','county','precinct','hd'
    ]).agg({'votes': 'sum'}).reset_index()
    hd_precinct_key['vote_fraction'] = hd_precinct_key.apply(lambda row: row['votes'] / (precinct_total_votes[row['key']] or 1), axis=1)

    sd_precinct_key = lege_votes_by_precinct.groupby([
        'key','county','precinct','sd'
    ]).agg({'votes': 'sum'}).reset_index()
    sd_precinct_key['vote_fraction'] = sd_precinct_key.apply(lambda row: row['votes'] / (precinct_total_votes[row['key']] or 1), axis=1)

    precinct_results_by_hd = precinct_results.merge(hd_precinct_key[['key','hd','vote_fraction']], on='key', how='left')
    precinct_results_by_sd = precinct_results.merge(sd_precinct_key[['key','sd','vote_fraction']], on='key', how='left')


    return precinct_results_by_hd, precinct_results_by_sd

precinct_results_by_hd, precinct_results_by_sd = apportion_precinct_results_to_legislative_districts(statewide_general_by_precinct_2024)

In [192]:
precinct_results_by_hd.head()

,election,type,race,key,county,precinct,party,candidate,votes,hd,vote_fraction
0,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 01 - D-CITY,BEAVERHEAD,PRECINCT 01 - D-CITY,Republican,DONALD J. TRUMP,230,HD 70,1.0
1,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 02 - D-RURAL,BEAVERHEAD,PRECINCT 02 - D-RURAL,Republican,DONALD J. TRUMP,652,HD 70,1.0
2,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 03 - D.RURAL,BEAVERHEAD,PRECINCT 03 - D.RURAL,Republican,DONALD J. TRUMP,698,HD 70,1.0
3,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 04 - LIMA,BEAVERHEAD,PRECINCT 04 - LIMA,Republican,DONALD J. TRUMP,174,HD 70,1.0
4,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 05 - DELL,BEAVERHEAD,PRECINCT 05 - DELL,Republican,DONALD J. TRUMP,125,HD 70,1.0


In [203]:
precinct_results_by_sd.head()

,election,type,race,key,county,precinct,party,candidate,votes,sd,vote_fraction
0,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 01 - D-CITY,BEAVERHEAD,PRECINCT 01 - D-CITY,Republican,DONALD J. TRUMP,230,SD 35,1.0
1,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 02 - D-RURAL,BEAVERHEAD,PRECINCT 02 - D-RURAL,Republican,DONALD J. TRUMP,652,SD 35,1.0
2,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 03 - D.RURAL,BEAVERHEAD,PRECINCT 03 - D.RURAL,Republican,DONALD J. TRUMP,698,SD 35,1.0
3,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 04 - LIMA,BEAVERHEAD,PRECINCT 04 - LIMA,Republican,DONALD J. TRUMP,174,SD 35,1.0
4,2024-gen,statewide,president,BEAVERHEAD-PRECINCT 05 - DELL,BEAVERHEAD,PRECINCT 05 - DELL,Republican,DONALD J. TRUMP,125,SD 35,1.0


In [205]:
# TODO: Does this merge take the first match or create new rows for multiple matches?

# precinctResultsApportionedHd = statewide_general_by_precinct_2024.merge(hd_precinct_key[['key','hd','vote_fraction']], on='key', how='left')
# precinctResultsApportionedSd = statewide_general_by_precinct_2024.merge(sd_precinct_key[['key','sd','vote_fraction']], on='key', how='left')

def parse_statewide_votes (precinctResults, district, raceLabel):
    #Filter precinct-level results to just results for a given race and lege district
    districtType = district[:2].lower()
    raceResultsByPrecinct = precinctResults[
        (precinctResults['race'] == raceLabel)
        & (precinctResults[districtType] == district)
    ].copy()
    raceResultsByPrecinct['votes_in_district'] = raceResultsByPrecinct['votes'] * raceResultsByPrecinct['vote_fraction']
    race_votes = raceResultsByPrecinct.groupby(['party', 'candidate']).agg({'votes_in_district': 'sum'}).reset_index()
    race_votes.rename({'votes_in_district': 'votes'}, axis=1, inplace=True)
    race_votes['race'] = district
    race_votes['party'] = race_votes['party'].str[:1]
    race_votes['votes'] =  race_votes['votes'].round().astype(int)
    total_votes = race_votes['votes'].sum()
    race_votes['percent'] = race_votes['votes'] / total_votes
    return race_votes

parse_statewide_votes(precinct_results_by_hd, 'HD 15', 'governor')

,party,candidate,votes,race,percent
0,D,RYAN BUSSE,2200,HD 15,0.526190
1,L,KAISER LEIB,128,HD 15,0.030615
2,R,GREG GIANFORTE,1853,HD 15,0.443195


In [ ]:
parse_statewide_votes(precinct_results_by_hd, 'HD 15', 'president')

,party,candidate,votes,race,percent
0,D,KAMALA D. HARRIS,2155,HD 15,0.514565
1,G,JILL STEIN,19,HD 15,0.004537
2,L,CHASE OLIVER,22,HD 15,0.005253
3,R,DONALD J. TRUMP,1871,HD 15,0.446753
4,W,ROBERT F. KENNEDY JR.,121,HD 15,0.028892


In [ ]:
# Total statewide
statewide_general_by_precinct_2024[statewide_general_by_precinct_2024['race'] == 'governor']\
    .groupby('candidate').agg({'votes': 'sum'}).sort_values('votes',ascending=False)

,votes
candidate,
GREG GIANFORTE,355609
RYAN BUSSE,233259
KAISER LEIB,15243


In [208]:
df = pd.DataFrame()
for district in lege_votes_by_precinct['hd'].unique():
    dfi = parse_statewide_votes(precinct_results_by_hd, district, 'governor')
    df = pd.concat([df, dfi])

df.groupby('candidate').agg({'votes': 'sum'}).sort_values('votes',ascending=False)

,votes
candidate,
GREG GIANFORTE,354569
RYAN BUSSE,232645
KAISER LEIB,15191


In [209]:
df = pd.DataFrame()
for district in lege_votes_by_precinct['sd'].unique():
    dfi = parse_statewide_votes(precinct_results_by_sd, district, 'governor')
    df = pd.concat([df, dfi])

df.groupby('candidate').agg({'votes': 'sum'}).sort_values('votes',ascending=False)

,votes
candidate,
GREG GIANFORTE,354569
RYAN BUSSE,232644
KAISER LEIB,15191
